In [23]:
import os
import re
import json
import sys
sys.path.append("../../")

import numpy as np
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data

import ThinLens.Elements as Elements
import ThinLens.Maps as Maps
from ThinLens.Models import F0D0Model, SIS18_Cell_minimal, SIS18_Cell, \
    SIS18_Lattice_minimal, SIS18_Lattice

import tools.madX

from cpymad.madx import TwissFailed

prepare model

In [24]:
dim = 6
slices = 4
quadSliceMultiplicity = 2
dtype = torch.double
device = torch.device("cpu")
outputPerElement = False  # exceeds outputAtBPM
outputAtBPM = True

# prepare models
Lattice = SIS18_Lattice

model = Lattice(dim=dim, slices=slices, quadSliceMultiplicity=quadSliceMultiplicity,
                dtype=dtype, cellsIdentical=False)

idealModel = Lattice(dim=dim, slices=slices, quadSliceMultiplicity=quadSliceMultiplicity, dtype=dtype,)

# load averaged weights
with open("/home/dylan/ThesisWorkspace/BayesianOptimization/dump/SIS18_Lattice_q1=0.3139705229205463,q2=-0.47506272187213366,step=0.0006653429815000447.json") as file:
    model.loadJSON(file)

In [25]:
def checkModelPerformance():
    # check max weight deviation
    weights, idealWeights = list(), list()
    for e in range(len(model.elements)):
        # only consider quadrupoles
        if not type(model.elements[e]) is Elements.Quadrupole:
            continue

        for m in range(len(model.elements[e].maps)):
            if model.elements[e].maps[m].weight.requires_grad:
                weights.append(model.elements[e].maps[m].weight.item())
                idealWeights.append(idealModel.elements[e].maps[m].weight.item())

    weights, idealWeights = np.array(weights), np.array(idealWeights)
    relWeightDiff = (weights - idealWeights) / idealWeights
    maxWeightDeviation = np.max(np.abs(relWeightDiff)) * 100  # convert to percent
    print("max. rel. deviation from ideal multipole strengths: {:.2f}%".format(maxWeightDeviation))

    # estimate beta-beating
    twiss, idealTwiss = tools.madX.twissTable(model.madX()), tools.madX.twissTable(idealModel.madX())

    beta, idealBeta = twiss["betx"], idealTwiss["betx"]

    relDiff = (beta - idealBeta) / idealBeta
    maxBetaBeat = np.max(np.abs(relDiff)) * 100  # convert to percent
    print("{:.2f}%".format(maxBetaBeat))

    # get max tune diff
    madXTunes = np.array(tools.madX.tune(model.madX()))
    madXTunesIdeal = np.array(tools.madX.tune(idealModel.madX())) 
    maxTuneDev = np.max(np.abs((madXTunesIdeal - madXTunes) / madXTunesIdeal),) * 100  # convert to percent

    q1, q2 = madXTunes[0], madXTunes[1]
    return q1, q2, maxWeightDeviation, maxBetaBeat, maxTuneDev

In [26]:
checkModelPerformance()

max. rel. deviation from ideal multipole strengths: 0.65%
1.46%


(4.229339325545347,
 3.303016197388341,
 0.6535665563226971,
 1.4645630877338598,
 0.07910213683718621)

check performance for samples within grid

In [ ]:
regexFName = re.compile("SIS18_Lattice_q1=(-?[\d.]*),q2=(-?[\d.]*).json")

benchmark = list()
for res in os.scandir("InitScan"):
    model.loadJSON(open(res, "r"))
    
    # benchmark
    try:
        bench = checkModelPerformance()
        benchmark.append([ *bench])
    except TwissFailed:
        continue

benchmark = np.array(benchmark)

max. rel. deviation from ideal multipole strengths: 1.76%
26.59%
max. rel. deviation from ideal multipole strengths: 0.74%
10.13%
max. rel. deviation from ideal multipole strengths: 2.25%
42.76%
max. rel. deviation from ideal multipole strengths: 2.09%
30.53%
max. rel. deviation from ideal multipole strengths: 0.71%
9.02%
max. rel. deviation from ideal multipole strengths: 1.54%
19.63%
max. rel. deviation from ideal multipole strengths: 7.00%
max. rel. deviation from ideal multipole strengths: 2.73%
39.52%
max. rel. deviation from ideal multipole strengths: 0.60%
4.38%
max. rel. deviation from ideal multipole strengths: 0.98%
13.05%
max. rel. deviation from ideal multipole strengths: 1.88%
27.72%
max. rel. deviation from ideal multipole strengths: 1.44%
21.30%
max. rel. deviation from ideal multipole strengths: 0.77%
9.64%
max. rel. deviation from ideal multipole strengths: 3.51%
60.00%
max. rel. deviation from ideal multipole strengths: 1.59%
23.30%
max. rel. deviation from ideal mult

In [ ]:
benchData = benchmark
benchData[:,0] = np.round(benchData[:,0])
benchData[:,1] = np.round(benchData[:,1])

# sort by first row
benchData = benchData[np.argsort(benchData[:,0])]